In [5]:
import os
import json
import glob
from PIL import Image

# Paths for UAVDT dataset
UAVDT_TRAIN_ANN_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\train\ann"
UAVDT_TRAIN_IMG_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\train\img"
UAVDT_TEST_ANN_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\test\ann"
UAVDT_TEST_IMG_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\test\img"

# Paths for VisDrone dataset
VISDRONE_IMG_PATH = r"C:\Users\jmdgo\Downloads\VisDrone2019-DET-train\VisDrone2019-DET-train\images"
VISDRONE_ANN_PATH = r"C:\Users\jmdgo\Downloads\VisDrone2019-DET-train\VisDrone2019-DET-train\annotations"

# COCO format template
def coco_template():
    return {"images": [], "annotations": [], "categories": []}

# Function to convert UAVDT annotations to COCO format
def convert_uavdt_to_coco(ann_path, img_path, output_json):
    coco_data = coco_template()
    category_mapping = {}
    annotation_id = 0

    for ann_file in glob.glob(os.path.join(ann_path, "*.json")):
        with open(ann_file, "r") as f:
            data = json.load(f)
        
        img_name = os.path.splitext(os.path.basename(ann_file))[0] + ".jpg"
        img_file = os.path.join(img_path, img_name)
        if not os.path.exists(img_file):
            continue
        img = Image.open(img_file)
        width, height = img.size
        
        image_id = len(coco_data["images"])
        coco_data["images"].append({
            "id": image_id,
            "file_name": img_name,
            "width": width,
            "height": height
        })
        
        for obj in data.get("objects", []):
            category = obj["category"]
            if category not in category_mapping:
                category_mapping[category] = len(category_mapping) + 1
                coco_data["categories"].append({"id": category_mapping[category], "name": category})
            
            bbox = obj["bbox"]  # Assuming format [x, y, width, height]
            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_mapping[category],
                "bbox": bbox,
                "area": bbox[2] * bbox[3],
                "iscrowd": 0
            })
            annotation_id += 1
    
    with open(output_json, "w") as f:
        json.dump(coco_data, f, indent=4)
    print(f"Saved: {output_json}")

# Function to convert VisDrone annotations to COCO format
def convert_visdrone_to_coco(img_path, ann_path, output_json):
    coco_data = coco_template()
    category_mapping = {}
    annotation_id = 0

    for img_file in glob.glob(os.path.join(img_path, "*.jpg")):
        img_name = os.path.basename(img_file)
        ann_file = os.path.join(ann_path, img_name.replace(".jpg", ".txt"))
        if not os.path.exists(ann_file):
            continue
        
        img = Image.open(img_file)
        width, height = img.size
        
        image_id = len(coco_data["images"])
        coco_data["images"].append({
            "id": image_id,
            "file_name": img_name,
            "width": width,
            "height": height
        })
        
        with open(ann_file, "r") as f:
            for line in f.readlines():  # ← Fix indentation here
                parts = line.strip().split(',')
                if len(parts) < 6:  # Ensure the line has enough values
                    continue

                x, y, w, h = map(float, parts[:4])  # Extract bbox coordinates
                class_id = int(parts[5])  # Get class_id (6th value)

                # Convert to COCO format
                if class_id not in category_mapping:
                    category_mapping[class_id] = len(category_mapping) + 1
                    coco_data["categories"].append({"id": category_mapping[class_id], "name": str(class_id)})

                coco_data["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": category_mapping[class_id],
                    "bbox": [x, y, w, h],
                    "area": w * h,
                    "iscrowd": 0
                })
                annotation_id += 1

    with open(output_json, "w") as f:
        json.dump(coco_data, f, indent=4)
    print(f"Saved: {output_json}")


# Create output directory
os.makedirs("output", exist_ok=True)

# Convert UAVDT
convert_uavdt_to_coco(UAVDT_TRAIN_ANN_PATH, UAVDT_TRAIN_IMG_PATH, "output/uavdt_train_coco.json")
convert_uavdt_to_coco(UAVDT_TEST_ANN_PATH, UAVDT_TEST_IMG_PATH, "output/uavdt_test_coco.json")

# Convert VisDrone
convert_visdrone_to_coco(VISDRONE_IMG_PATH, VISDRONE_ANN_PATH, "output/visdrone_coco.json")


Saved: output/uavdt_train_coco.json
Saved: output/uavdt_test_coco.json
Saved: output/visdrone_coco.json


In [3]:
import os

UAVDT_TRAIN_IMG_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\train\img"

image_files = os.listdir(UAVDT_TRAIN_IMG_PATH)
print("Example image filenames:", image_files[:10])  # Print first 10 filenames


Example image filenames: ['M0101_img000001.jpg', 'M0101_img000002.jpg', 'M0101_img000003.jpg', 'M0101_img000004.jpg', 'M0101_img000005.jpg', 'M0101_img000006.jpg', 'M0101_img000007.jpg', 'M0101_img000008.jpg', 'M0101_img000009.jpg', 'M0101_img000010.jpg']


In [8]:
import os

train_images = set(os.listdir(UAVDT_TRAIN_IMG_PATH))
test_images = set(os.listdir(UAVDT_TEST_IMG_PATH))

print(f"Train images: {len(train_images)}")
print(f"Test images: {len(test_images)}")

extra_test = test_images - train_images
print(f"Extra images in test: {len(extra_test)}")

Train images: 24143
Test images: 53676
Extra images in test: 53676


In [14]:
import os
import json
import glob

# Paths for UAVDT dataset
UAVDT_TRAIN_ANN_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\train\ann"
UAVDT_TRAIN_IMG_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\train\img"
UAVDT_TEST_ANN_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\test\ann"
UAVDT_TEST_IMG_PATH = r"C:\Users\jmdgo\Downloads\UAVDT\test\img"

# COCO format template
def coco_template():
    return {"images": [], "annotations": [], "categories": []}

# Function to convert points to COCO bbox format
def convert_points_to_bbox(points):
    if "exterior" not in points or len(points["exterior"]) != 2:
        return None  # Invalid format
    x1, y1 = points["exterior"][0]
    x2, y2 = points["exterior"][1]
    x, y = x1, y1
    width, height = x2 - x1, y2 - y1
    if width <= 0 or height <= 0:
        return None  # Invalid bbox
    return [x, y, width, height]

# Function to convert UAVDT annotations to COCO format
def convert_uavdt_to_coco(ann_path, img_path, output_json):
    coco_data = coco_template()
    category_mapping = {}
    annotation_id = 0

    # Get all image files in the dataset
    image_files = {os.path.basename(f) for f in glob.glob(os.path.join(img_path, "*"))}

    # Get all annotation files
    ann_files = glob.glob(os.path.join(ann_path, "*.json"))
    print(f"Found {len(ann_files)} annotation files in {ann_path}")

    if len(ann_files) == 0:
        print(f"No annotation files found in {ann_path}")
        return

    processed_images = 0
    skipped_images = 0
    total_annotations = 0

    for ann_file in ann_files:
        with open(ann_file, "r") as f:
            data = json.load(f)

        if "size" not in data or "objects" not in data:
            skipped_images += 1
            continue

        # Extract image filename correctly
        img_name = os.path.splitext(os.path.basename(ann_file))[0]
        if not img_name.endswith(".jpg"):
            img_name += ".jpg"

        if img_name not in image_files:
            print(f"Image {img_name} not found in {img_path}, skipping.")
            skipped_images += 1
            continue

        # Get image size
        width, height = data["size"].get("width", 0), data["size"].get("height", 0)
        if width <= 0 or height <= 0:
            skipped_images += 1
            continue

        # Add image to COCO dataset
        image_id = len(coco_data["images"]) + 1
        coco_data["images"].append({
            "id": image_id,
            "file_name": img_name,
            "width": width,
            "height": height
        })
        processed_images += 1

        # Process annotations
        for obj in data["objects"]:
            category = obj.get("classTitle", "unknown")  # Extract category
            bbox = convert_points_to_bbox(obj.get("points", {}))  # Convert points to bbox
            if not category or bbox is None:
                continue

            if category not in category_mapping:
                category_mapping[category] = len(category_mapping) + 1
                coco_data["categories"].append({"id": category_mapping[category], "name": category})

            # Add annotation
            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_mapping[category],
                "bbox": bbox,
                "area": bbox[2] * bbox[3],
                "iscrowd": 0
            })
            annotation_id += 1
            total_annotations += 1

    # Save the COCO formatted annotations
    output_path = os.path.abspath("output")
    os.makedirs(output_path, exist_ok=True)
    output_file = os.path.join(output_path, output_json)

    with open(output_file, "w") as f:
        json.dump(coco_data, f, indent=4)
    
    print(f"\nSummary for {output_json}:")
    print(f"- Processed Images: {processed_images}")
    print(f"- Skipped Images: {skipped_images}")
    print(f"- Total Annotations: {total_annotations}")
    print(f"Saved: {output_file}\n")

# Convert UAVDT dataset
convert_uavdt_to_coco(UAVDT_TRAIN_ANN_PATH, UAVDT_TRAIN_IMG_PATH, "uavdt_train_coco.json")
convert_uavdt_to_coco(UAVDT_TEST_ANN_PATH, UAVDT_TEST_IMG_PATH, "uavdt_test_coco.json")

Found 24143 annotation files in C:\Users\jmdgo\Downloads\UAVDT\train\ann

Summary for uavdt_train_coco.json:
- Processed Images: 24143
- Skipped Images: 0
- Total Annotations: 422911
Saved: C:\Users\jmdgo\Documents\CV\output\uavdt_train_coco.json

Found 53676 annotation files in C:\Users\jmdgo\Downloads\UAVDT\test\ann

Summary for uavdt_test_coco.json:
- Processed Images: 53676
- Skipped Images: 0
- Total Annotations: 412968
Saved: C:\Users\jmdgo\Documents\CV\output\uavdt_test_coco.json



In [2]:
import json
import os
from tqdm import tqdm

# Input/Output paths
INPUT_JSON = r"C:\Users\jmdgo\Documents\CV\NOMAD annotations.json"
OUTPUT_JSON = r"C:\Users\jmdgo\Documents\CV\nomad_coco.json"

# Category mapping — assuming category_id 0 = person
CATEGORY_MAPPING = {
    0: {"id": 1, "name": "person"}
}

# Load NOMAD annotations
with open(INPUT_JSON, "r") as f:
    nomad_data = json.load(f)

# Initialize COCO structure
coco_format = {
    "images": [],
    "annotations": [],
    "categories": list(CATEGORY_MAPPING.values())
}

annotation_id = 1
for image_id, entry in enumerate(tqdm(nomad_data), start=1):
    # Add image metadata
    coco_format["images"].append({
        "id": image_id,
        "file_name": entry["file_name"],
        "height": entry["height"],
        "width": entry["width"]
    })

    for ann in entry["annotations"]:
        bbox = ann["bbox"]
        category_id = ann["category_id"]  # Assuming always 0
        coco_format["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": CATEGORY_MAPPING[category_id]["id"],
            "bbox": bbox,
            "area": bbox[2] * bbox[3],
            "iscrowd": ann.get("iscrowd", 0)
        })
        annotation_id += 1

# Save COCO-style JSON
with open(OUTPUT_JSON, "w") as f:
    json.dump(coco_format, f, indent=2)

print(f"✅ COCO-format annotations saved at:\n{OUTPUT_JSON}")

100%|████████████████████████████████████████████████████████████████████████| 42825/42825 [00:00<00:00, 208547.19it/s]


✅ COCO-format annotations saved at:
C:\Users\jmdgo\Documents\CV\nomad_coco.json
